# AIB to Bambora SF Cost Comparison Application

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Import the Necessary Libraries
Below are the libraries required to assemble the project.

In [2]:
import sys
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import cx_Oracle
conn=cx_Oracle.connect('djohnson','RhiBae01!','ODSPEU1.ams')

In [4]:
ab=pd.read_sql('''
SELECT 
bb.BAMBORA_REF
,TO_CHAR(bb.SETTLEMENT_DATE, 'MM') AS MONTH
,TO_CHAR(bb.SETTLEMENT_DATE, 'DD') AS DAY
,TO_CHAR(bb.SETTLEMENT_DATE, 'YYYY') AS YEAR
,TO_CHAR(bb.SETTLEMENT_DATE,'YYYY-MM') AS PERIOD
,bb.CONTRACT_ID,bb.ISSUER_COUNTRY_NAME, bb.MERCHANT_COUNTRY_NAME,bb.TRANSACTION_TYPE,bb.CARD_SCHEME,bb.CREDIT_DEBIT,bb.CONSUMER_CORPORATE,bb.REGIONALITY
,bb.SF_FIXED_FEE,bb.SF_PERC_FEE,bb.TRANSACTION_TYPE AS B_TRANSACTION_TYPE,bb.CONSUMER_CORPORATE AS C_TYPE
,bb.VOLUME AS VOLUME,ROUND(bb.TRANSACTION_AMOUNT_EUR) AS FLOW,bb.SCHEME_TOTAL_FEE_EUR AS SF,bb.TRANSACTION_AMOUNT_EUR
,BB.INTERCHANGE_TOTAL_FEE_EUR,BB.INTERCHANGE_TOTAL_FEE_EUR/bb.TRANSACTION_AMOUNT_EUR AS IC_BPS,BB.SCHEME_TOTAL_FEE_EUR/bb.TRANSACTION_AMOUNT_EUR AS SF_BPS
FROM fdwo.acquirer_bambora_cost bb
WHERE bb.TRANSACTION_TYPE='TRANSACTION'
AND bb.SETTLEMENT_DATE BETWEEN TO_DATE('01-01-2018','MM-DD-YYYY') AND TO_DATE('02-22-2018','MM-DD-YYYY')
''',conn)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: ORA-03113: end-of-file on communication channel
Process ID: 330225
Session ID: 1455 Serial number: 42281

## AIB Import

In [3]:
ab=pd.read_csv('d:\\djohnson\\Desktop\\aib.txt',sep=';')

## BB Import

In [33]:
bb=pd.read_csv('d:\\djohnson\\Desktop\\bb.txt',sep=';')

## Create a Working Copy of the Bambora Data

In [34]:
bb1=bb.copy() #create working copy of the Bambora data

## Rename Columns to Allow for Merge

In [36]:
bb1=bb1.rename(columns={'ISSUER_COUNTRY_NAME':'ISSUER_COUNTRY','MERCHANT_COUNTRY_NAME':'CLEARING_COUNTRY_NAME','CONSUMER_CORPORATE':'CARD_TYPE','CARD_SCHEME':'CARD_BRAND'})

In [39]:
print bb1.head()

               BAMBORA_REF  MONTH  DAY  YEAR   PERIOD  CONTRACT_ID  \
0  74541098005025577288894      1    5  2018  2018-01         4235   
1  74541098005025577272203      1    5  2018  2018-01         4235   
2  25481048005025577280940      1    5  2018  2018-01         4235   
3  25481048005025577280635      1    5  2018  2018-01         4235   
4  74541098005025577277210      1    5  2018  2018-01         4235   

              ISSUER_COUNTRY CLEARING_COUNTRY_NAME TRANSACTION_TYPE  \
0                    Ukraine               Germany      TRANSACTION   
1                  Argentina               Germany      TRANSACTION   
2                    Ukraine               Germany      TRANSACTION   
3  Taiwan, Province of China               Germany      TRANSACTION   
4                     Israel               Germany      TRANSACTION   

   CARD_BRAND    ...    SF_PERC_FEE B_TRANSACTION_TYPE    C_TYPE  VOLUME  \
0        Visa    ...       0.005669        TRANSACTION  Consumer       1   


## Indentify Congruent Lanes

In [13]:
bb1_contracts=bb1[['CONTRACT_ID','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME','CARD_BRAND','CREDIT_DEBIT','CARD_TYPE']].drop_duplicates()

### Remove Excess Naming

In [14]:
ab=ab.drop(['TRANSACTION_TYPE','TRANSACTION_DATE'],axis=1)

In [15]:
print ab.head()

   CONTRACT_ID       ISSUER_COUNTRY  CARD_BRAND CLEARING_COUNTRY_NAME  FLOW  \
0         1400               Panama        Visa                Poland    11   
1         4382  Korea - Republic of  MasterCard               Hungary    92   
2           12                Spain        Visa                Poland     2   
3           12               Poland        Visa                Poland     2   
4         4382               Taiwan        Visa               Hungary   220   

         SF    IC_EUR  INTERCHANGE_RATE_AMOUNT  INTERCHANGE_RATE_PERC  \
0  0.129730  0.172904                      0.0                   1.60   
1  0.641318  1.476270                      0.0                   1.60   
2  0.020155  0.003100                      0.0                   0.20   
3  0.020157  0.003140                      0.0                   0.20   
4  1.274438  4.313791                      0.0                   1.97   

     IC_BPS CREDIT_DEBIT CARD_TYPE  
0  0.015719       Credit  Consumer  
1  0.016046 

## Attach AIB Lanes to Active Bambora Lanes 
This is a left join that will show all active Bambora lanes on only those AIB lanes where there is congruence.

In [16]:
ab_df=pd.merge(bb1_contracts,ab,on=['CONTRACT_ID','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME','CARD_BRAND','CREDIT_DEBIT','CARD_TYPE'],how='left')

In [17]:
bb1_contracts=None #clear the merge data
ab=None #clear the merge data

### Only show those lanes where there is activity for AIB and Bambora
This is done but filtering to only those results that have scheme fee information

In [18]:
ab_df=ab_df[(ab_df['SF']>0)].drop_duplicates()

### Only Show the Median aggregated by lane, contract ID, Credit/Debit and Card type, Flow

In [19]:
ab_df=ab_df.groupby(['CONTRACT_ID','ISSUER_COUNTRY','CARD_BRAND','CLEARING_COUNTRY_NAME','CREDIT_DEBIT','CARD_TYPE','FLOW'],as_index=False)['SF'].median()

In [20]:
print ab_df.head()

   CONTRACT_ID      ISSUER_COUNTRY CARD_BRAND CLEARING_COUNTRY_NAME  \
0         1946  Russian Federation       Visa            Luxembourg   
1         1946  Russian Federation       Visa            Luxembourg   
2         1946  Russian Federation       Visa            Luxembourg   
3         1946  Russian Federation       Visa            Luxembourg   
4         1946  Russian Federation       Visa            Luxembourg   

  CREDIT_DEBIT CARD_TYPE  FLOW        SF  
0        Debit  Consumer   0.0  0.080190  
1        Debit  Consumer   1.0  0.080962  
2        Debit  Consumer   2.0  0.081890  
3        Debit  Consumer   3.0  0.086201  
4        Debit  Consumer   4.0  0.090757  


In [21]:
ab_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 8 columns):
CONTRACT_ID              100 non-null int64
ISSUER_COUNTRY           100 non-null object
CARD_BRAND               100 non-null object
CLEARING_COUNTRY_NAME    100 non-null object
CREDIT_DEBIT             100 non-null object
CARD_TYPE                100 non-null object
FLOW                     100 non-null float64
SF                       100 non-null float64
dtypes: float64(2), int64(1), object(5)
memory usage: 7.0+ KB


## Setup the Bambora Baseline Lanes, Flows, and SFs

In [22]:
base=ab_df.drop(['FLOW','SF'],axis=1).drop_duplicates()

In [23]:
bbbase=pd.merge(base,bb1,on=['CONTRACT_ID','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME','CARD_BRAND','CREDIT_DEBIT','CARD_TYPE'],how='left')

In [24]:
bbbase.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_BRAND,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,BAMBORA_REF,MONTH,DAY,YEAR,...,SF_FIXED_FEE,SF_PERC_FEE,B_TRANSACTION_TYPE,C_TYPE,VOLUME,FLOW,SF,TRANSACTION_AMOUNT_EUR,INTERCHANGE_TOTAL_FEE_EUR,IC_BPS
0,1946,Russian Federation,Visa,Luxembourg,Debit,Consumer,74541097364025519785764,12,30,2017,...,0.072944,0.005602,TRANSACTION,Consumer,1,13,0.133969,13.124052,0.208534,0.015889
1,1946,Russian Federation,Visa,Luxembourg,Debit,Consumer,74541097364025519785756,12,30,2017,...,0.072944,0.005603,TRANSACTION,Consumer,1,12,0.130193,12.445249,0.222467,0.017876


In [25]:
bbbase.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 23 columns):
CONTRACT_ID                  2 non-null int64
ISSUER_COUNTRY               2 non-null object
CARD_BRAND                   2 non-null object
CLEARING_COUNTRY_NAME        2 non-null object
CREDIT_DEBIT                 2 non-null object
CARD_TYPE                    2 non-null object
BAMBORA_REF                  2 non-null object
MONTH                        2 non-null int64
DAY                          2 non-null int64
YEAR                         2 non-null int64
PERIOD                       2 non-null object
TRANSACTION_TYPE             2 non-null object
REGIONALITY                  2 non-null object
SF_FIXED_FEE                 2 non-null float64
SF_PERC_FEE                  2 non-null float64
B_TRANSACTION_TYPE           2 non-null object
C_TYPE                       2 non-null object
VOLUME                       2 non-null int64
FLOW                         2 non-null int64
SF        

### Link AIB and Bambora

In [26]:
ab_bb_comp=pd.merge(bbbase,ab_df,on=['CONTRACT_ID','ISSUER_COUNTRY','CARD_BRAND','CLEARING_COUNTRY_NAME','CREDIT_DEBIT','CARD_TYPE','FLOW'],how='left',suffixes=('_BB','_AVG_AB'))

In [27]:
ab_bb_comp.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_BRAND,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,BAMBORA_REF,MONTH,DAY,YEAR,...,SF_PERC_FEE,B_TRANSACTION_TYPE,C_TYPE,VOLUME,FLOW,SF_BB,TRANSACTION_AMOUNT_EUR,INTERCHANGE_TOTAL_FEE_EUR,IC_BPS,SF_AVG_AB
0,1946,Russian Federation,Visa,Luxembourg,Debit,Consumer,74541097364025519785764,12,30,2017,...,0.005602,TRANSACTION,Consumer,1,13,0.133969,13.124052,0.208534,0.015889,0.140335
1,1946,Russian Federation,Visa,Luxembourg,Debit,Consumer,74541097364025519785756,12,30,2017,...,0.005603,TRANSACTION,Consumer,1,12,0.130193,12.445249,0.222467,0.017876,0.134877


## Insert the Signal logic

In [28]:
ab_bb_comp['SF_DIFF_%']=(ab_bb_comp['SF_BB']-ab_bb_comp['SF_AVG_AB'])/ab_bb_comp['SF_AVG_AB']

ab_bb_comp['SIGNAL']=''
ab_bb_comp['SIGNAL'][(ab_bb_comp['SF_DIFF_%']>.05)]='Signal'

C:\Users\djohnson\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [29]:
ab_bb_comp.head()

,CONTRACT_ID,ISSUER_COUNTRY,CARD_BRAND,CLEARING_COUNTRY_NAME,CREDIT_DEBIT,CARD_TYPE,BAMBORA_REF,MONTH,DAY,YEAR,...,C_TYPE,VOLUME,FLOW,SF_BB,TRANSACTION_AMOUNT_EUR,INTERCHANGE_TOTAL_FEE_EUR,IC_BPS,SF_AVG_AB,SF_DIFF_%,SIGNAL
0,1946,Russian Federation,Visa,Luxembourg,Debit,Consumer,74541097364025519785764,12,30,2017,...,Consumer,1,13,0.133969,13.124052,0.208534,0.015889,0.140335,-0.045361,
1,1946,Russian Federation,Visa,Luxembourg,Debit,Consumer,74541097364025519785756,12,30,2017,...,Consumer,1,12,0.130193,12.445249,0.222467,0.017876,0.134877,-0.034730,


### Final Output

In [30]:
ab_bb_comp=ab_bb_comp[['BAMBORA_REF','TRANSACTION_TYPE','REGIONALITY','CONTRACT_ID','MONTH','DAY','YEAR','PERIOD'
                       ,'CARD_BRAND','CARD_TYPE','CREDIT_DEBIT','ISSUER_COUNTRY','CLEARING_COUNTRY_NAME'
                       ,'FLOW','SF_FIXED_FEE','SF_PERC_FEE','SF_BB','SF_AVG_AB','SF_DIFF_%','SIGNAL']]

In [31]:
ab_bb_comp.head()

,BAMBORA_REF,TRANSACTION_TYPE,REGIONALITY,CONTRACT_ID,MONTH,DAY,YEAR,PERIOD,CARD_BRAND,CARD_TYPE,CREDIT_DEBIT,ISSUER_COUNTRY,CLEARING_COUNTRY_NAME,FLOW,SF_FIXED_FEE,SF_PERC_FEE,SF_BB,SF_AVG_AB,SF_DIFF_%,SIGNAL
0,74541097364025519785764,TRANSACTION,Interregional,1946,12,30,2017,2017-12,Visa,Consumer,Debit,Russian Federation,Luxembourg,13,0.072944,0.005602,0.133969,0.140335,-0.045361,
1,74541097364025519785756,TRANSACTION,Interregional,1946,12,30,2017,2017-12,Visa,Consumer,Debit,Russian Federation,Luxembourg,12,0.072944,0.005603,0.130193,0.134877,-0.034730,


In [32]:
print 'total transactions'
print ab_bb_comp['BAMBORA_REF'].count()
print 'total signals'
print ab_bb_comp['BAMBORA_REF'][(ab_bb_comp['SIGNAL']=='Signal')].count()

total transactions
2
total signals
0


In [46]:
ab_bb_comp.to_csv('d:\\djohnson\\Desktop\\ab_bb_comp.csv')